## Model training and Evaluation

In [1]:
# import necessory packages
import pandas as pd
import numpy as np
import os
import time
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler,RobustScaler
from sklearn.compose import ColumnTransformer
import tensorflow
import keras_tuner
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingRegressor
from kerastuner.tuners import RandomSearch
from tensorflow.keras import optimizers
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

C:\Users\anike\AppData\Local\Temp\ipykernel_14400\1337451595.py:20: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [2]:
# import cleaned data
data = pd.read_csv('cleaned_data.csv')

In [3]:
df = data.copy()

In [4]:
#pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)

In [5]:
# checking shape of dataset
df.shape

(1016, 15)

In [6]:
df.isnull().sum()

brand_names           0
price                 0
score               420
processor brand       0
processor type        4
processor gen       101
type of core         23
no of threads         0
Ram                   0
ram type              1
storage type          0
storage capacity      8
ppi                  34
OS                    0
Warranty             86
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1016 entries, 0 to 1015
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   brand_names       1016 non-null   object 
 1   price             1016 non-null   int64  
 2   score             596 non-null    float64
 3   processor brand   1016 non-null   object 
 4   processor type    1012 non-null   object 
 5   processor gen     915 non-null    float64
 6   type of core      993 non-null    float64
 7   no of threads     1016 non-null   float64
 8   Ram               1016 non-null   int64  
 9   ram type          1015 non-null   object 
 10  storage type      1016 non-null   object 
 11  storage capacity  1008 non-null   float64
 12  ppi               982 non-null    float64
 13  OS                1016 non-null   object 
 14  Warranty          930 non-null    float64
dtypes: float64(7), int64(2), object(6)
memory usage: 119.2+ KB


In [8]:
num_col = [col for col in df if df[col].dtypes!='O']
cat_col = [col for col in df if df[col].dtypes=='O']

In [9]:
num_col

['price',
 'score',
 'processor gen',
 'type of core',
 'no of threads',
 'Ram',
 'storage capacity',
 'ppi',
 'Warranty']

In [10]:
num_col[1:]

['score',
 'processor gen',
 'type of core',
 'no of threads',
 'Ram',
 'storage capacity',
 'ppi',
 'Warranty']

In [11]:
cat_col

['brand_names',
 'processor brand',
 'processor type',
 'ram type',
 'storage type',
 'OS']

In [12]:
X = df.drop('price',axis=1)
y = df['price']

In [13]:
numeric_preprocessor = Pipeline(steps=[
                                        ("scaler", RobustScaler()),
                                        ("imputation_mean",SimpleImputer(missing_values=np.nan, strategy="mean"))
                                      ])

categorical_preprocessor = Pipeline(steps=[
                                            ("imputer",SimpleImputer(strategy="constant")),
                                            ("encoder", OneHotEncoder(sparse=False,drop='first',handle_unknown="ignore")),
                                        ])

preprocessor = ColumnTransformer([
                                    ("categorical", categorical_preprocessor, cat_col),
                                    ("numerical", numeric_preprocessor,  num_col[1:]),
                                ])

In [14]:
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 10)

In [15]:
X_train.shape

(812, 14)

In [16]:
# Transformed the preprocessed data
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [17]:
X_train.shape

(812, 44)

In [82]:
# model training
model = Sequential([
                    Dense(256,kernel_initializer = 'he_uniform',activation='relu',input_dim = 44),
                    Dense(128,kernel_initializer = 'he_uniform',activation='relu'),
                    Dense(128,kernel_initializer = 'he_uniform',activation='relu'),
                    Dense(96,kernel_initializer = 'he_uniform',activation='relu'),
                    Dense(1,activation = 'linear')
                    ])

In [83]:
# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

In [84]:
# Combine the preprocessor and model into a single pipeline
regressor = Pipeline(steps=[('model', model)])

In [85]:
# Fit the pipeline to the training data
model_history = regressor.fit(X_train, y_train, model__epochs=500, model__batch_size=32,model__validation_split=0.15)

Epoch 1/500
22/22 [==============================] - 2s 16ms/step - loss: 10586697728.0000 - val_loss: 7981934592.0000
Epoch 2/500
22/22 [==============================] - 0s 5ms/step - loss: 8491916800.0000 - val_loss: 6130285568.0000
Epoch 3/500
22/22 [==============================] - 0s 6ms/step - loss: 6457804800.0000 - val_loss: 6022047232.0000
Epoch 4/500
22/22 [==============================] - 0s 6ms/step - loss: 6137444352.0000 - val_loss: 5971638784.0000
Epoch 5/500
22/22 [==============================] - 0s 6ms/step - loss: 6168901632.0000 - val_loss: 5934956032.0000
Epoch 6/500
22/22 [==============================] - 0s 5ms/step - loss: 6007421952.0000 - val_loss: 5850868224.0000
Epoch 7/500
22/22 [==============================] - 0s 6ms/step - loss: 6067443200.0000 - val_loss: 5766993920.0000
Epoch 8/500
22/22 [==============================] - 0s 6ms/step - loss: 5880813056.0000 - val_loss: 5713531392.0000
Epoch 9/500
22/22 [==============================] - 0s 5ms/st

Epoch 71/500
22/22 [==============================] - 0s 6ms/step - loss: 919451840.0000 - val_loss: 705321664.0000
Epoch 72/500
22/22 [==============================] - 0s 5ms/step - loss: 937838016.0000 - val_loss: 729993152.0000
Epoch 73/500
22/22 [==============================] - 0s 5ms/step - loss: 914582976.0000 - val_loss: 747436352.0000
Epoch 74/500
22/22 [==============================] - 0s 5ms/step - loss: 922052800.0000 - val_loss: 716935808.0000
Epoch 75/500
22/22 [==============================] - 0s 5ms/step - loss: 950055616.0000 - val_loss: 726671872.0000
Epoch 76/500
22/22 [==============================] - 0s 5ms/step - loss: 1072996736.0000 - val_loss: 706809088.0000
Epoch 77/500
22/22 [==============================] - 0s 5ms/step - loss: 990000576.0000 - val_loss: 686836032.0000
Epoch 78/500
22/22 [==============================] - 0s 5ms/step - loss: 980750784.0000 - val_loss: 734973376.0000
Epoch 79/500
22/22 [==============================] - 0s 5ms/step - los

22/22 [==============================] - 0s 6ms/step - loss: 813032192.0000 - val_loss: 648929664.0000
Epoch 142/500
22/22 [==============================] - 0s 5ms/step - loss: 791815424.0000 - val_loss: 659853312.0000
Epoch 143/500
22/22 [==============================] - 0s 5ms/step - loss: 788413888.0000 - val_loss: 630748032.0000
Epoch 144/500
22/22 [==============================] - 0s 5ms/step - loss: 886962112.0000 - val_loss: 630549888.0000
Epoch 145/500
22/22 [==============================] - 0s 5ms/step - loss: 799401472.0000 - val_loss: 641238784.0000
Epoch 146/500
22/22 [==============================] - 0s 5ms/step - loss: 801024384.0000 - val_loss: 642449600.0000
Epoch 147/500
22/22 [==============================] - 0s 5ms/step - loss: 801622592.0000 - val_loss: 645132480.0000
Epoch 148/500
22/22 [==============================] - 0s 5ms/step - loss: 825632704.0000 - val_loss: 638051136.0000
Epoch 149/500
22/22 [==============================] - 0s 5ms/step - loss: 783

22/22 [==============================] - 0s 6ms/step - loss: 799754688.0000 - val_loss: 787704384.0000
Epoch 212/500
22/22 [==============================] - 0s 5ms/step - loss: 810928000.0000 - val_loss: 666828736.0000
Epoch 213/500
22/22 [==============================] - 0s 5ms/step - loss: 734398400.0000 - val_loss: 679273408.0000
Epoch 214/500
22/22 [==============================] - 0s 5ms/step - loss: 771762176.0000 - val_loss: 691696192.0000
Epoch 215/500
22/22 [==============================] - 0s 5ms/step - loss: 733800704.0000 - val_loss: 659341248.0000
Epoch 216/500
22/22 [==============================] - 0s 5ms/step - loss: 732807232.0000 - val_loss: 706766656.0000
Epoch 217/500
22/22 [==============================] - 0s 5ms/step - loss: 746122816.0000 - val_loss: 675338816.0000
Epoch 218/500
22/22 [==============================] - 0s 5ms/step - loss: 763892992.0000 - val_loss: 677847296.0000
Epoch 219/500
22/22 [==============================] - 0s 5ms/step - loss: 720

22/22 [==============================] - 0s 5ms/step - loss: 847465024.0000 - val_loss: 670665536.0000
Epoch 282/500
22/22 [==============================] - 0s 5ms/step - loss: 896250368.0000 - val_loss: 665600128.0000
Epoch 283/500
22/22 [==============================] - 0s 5ms/step - loss: 711540416.0000 - val_loss: 635275200.0000
Epoch 284/500
22/22 [==============================] - 0s 5ms/step - loss: 683568768.0000 - val_loss: 671301952.0000
Epoch 285/500
22/22 [==============================] - 0s 5ms/step - loss: 713665408.0000 - val_loss: 657195008.0000
Epoch 286/500
22/22 [==============================] - 0s 5ms/step - loss: 703358464.0000 - val_loss: 660709760.0000
Epoch 287/500
22/22 [==============================] - 0s 5ms/step - loss: 692693312.0000 - val_loss: 661648640.0000
Epoch 288/500
22/22 [==============================] - 0s 5ms/step - loss: 685218944.0000 - val_loss: 656543680.0000
Epoch 289/500
22/22 [==============================] - 0s 5ms/step - loss: 681

22/22 [==============================] - 0s 5ms/step - loss: 648596352.0000 - val_loss: 666714752.0000
Epoch 352/500
22/22 [==============================] - 0s 5ms/step - loss: 641004160.0000 - val_loss: 703196736.0000
Epoch 353/500
22/22 [==============================] - 0s 5ms/step - loss: 637931200.0000 - val_loss: 689653824.0000
Epoch 354/500
22/22 [==============================] - 0s 5ms/step - loss: 646260160.0000 - val_loss: 672701248.0000
Epoch 355/500
22/22 [==============================] - 0s 5ms/step - loss: 697575616.0000 - val_loss: 650011776.0000
Epoch 356/500
22/22 [==============================] - 0s 5ms/step - loss: 708170496.0000 - val_loss: 656178816.0000
Epoch 357/500
22/22 [==============================] - 0s 5ms/step - loss: 633414912.0000 - val_loss: 659052736.0000
Epoch 358/500
22/22 [==============================] - 0s 5ms/step - loss: 647027072.0000 - val_loss: 652126464.0000
Epoch 359/500
22/22 [==============================] - 0s 5ms/step - loss: 703

22/22 [==============================] - 0s 7ms/step - loss: 572116352.0000 - val_loss: 683284544.0000
Epoch 422/500
22/22 [==============================] - 0s 6ms/step - loss: 573981568.0000 - val_loss: 660954688.0000
Epoch 423/500
22/22 [==============================] - 0s 5ms/step - loss: 564995328.0000 - val_loss: 689914304.0000
Epoch 424/500
22/22 [==============================] - 0s 5ms/step - loss: 595600192.0000 - val_loss: 666006720.0000
Epoch 425/500
22/22 [==============================] - 0s 5ms/step - loss: 576140928.0000 - val_loss: 686418048.0000
Epoch 426/500
22/22 [==============================] - 0s 5ms/step - loss: 588345280.0000 - val_loss: 669544832.0000
Epoch 427/500
22/22 [==============================] - 0s 5ms/step - loss: 578140864.0000 - val_loss: 694643712.0000
Epoch 428/500
22/22 [==============================] - 0s 5ms/step - loss: 608303360.0000 - val_loss: 683918976.0000
Epoch 429/500
22/22 [==============================] - 0s 5ms/step - loss: 636

22/22 [==============================] - 0s 7ms/step - loss: 535263616.0000 - val_loss: 641036096.0000
Epoch 492/500
22/22 [==============================] - 0s 6ms/step - loss: 531945792.0000 - val_loss: 657495808.0000
Epoch 493/500
22/22 [==============================] - 0s 6ms/step - loss: 503733440.0000 - val_loss: 709571904.0000
Epoch 494/500
22/22 [==============================] - 0s 6ms/step - loss: 520712832.0000 - val_loss: 689143424.0000
Epoch 495/500
22/22 [==============================] - 0s 6ms/step - loss: 552609280.0000 - val_loss: 680865920.0000
Epoch 496/500
22/22 [==============================] - 0s 5ms/step - loss: 549508288.0000 - val_loss: 657448320.0000
Epoch 497/500
22/22 [==============================] - 0s 6ms/step - loss: 544639168.0000 - val_loss: 758041856.0000
Epoch 498/500
22/22 [==============================] - 0s 6ms/step - loss: 588654784.0000 - val_loss: 798253056.0000
Epoch 499/500
22/22 [==============================] - 0s 6ms/step - loss: 578

In [86]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 256)               11520     
                                                                 
 dense_24 (Dense)            (None, 128)               32896     
                                                                 
 dense_25 (Dense)            (None, 128)               16512     
                                                                 
 dense_26 (Dense)            (None, 96)                12384     
                                                                 
 dense_27 (Dense)            (None, 1)                 97        
                                                                 
Total params: 73,409
Trainable params: 73,409
Non-trainable params: 0
_________________________________________________________________


In [87]:
# Evaluate the model on the test data
loss= model.evaluate(X_test, y_test)
loss

7/7 [==============================] - 0s 4ms/step - loss: 460534656.0000


460534656.0

In [88]:
# Make predictions on the test data
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

7/7 [==============================] - 0s 4ms/step


In [89]:
# Calculate the metrics
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

# Print the metrics
print("RMSE_train: ", rmse_train)
print("R2_train: ", r2_train)
print('**************************')
print("RMSE_test: ", rmse_test)
print("R2_test: ", r2_test)

RMSE_train:  23495.10960978838
R2_train:  0.8558846514781566
**************************
RMSE_test:  21460.07180326316
R2_test:  0.8320247562614236


In [90]:
y_pred_test[0:6]

array([[ 44698.043],
       [ 71519.33 ],
       [ 51958.43 ],
       [203424.05 ],
       [ 99147.695],
       [ 19040.898]], dtype=float32)

In [91]:
y_test[0:6]

998     56990
304     63350
271     51018
625    159990
951     68999
353     21833
Name: price, dtype: int64

In [92]:
#Hyperaparameter Tuning

In [93]:
def saveModel_path(model_dir="SAVED_MODELS"):
    os.makedirs(model_dir, exist_ok=True)
    fileName = time.strftime("Model_%Y_%m_%d_%H_%M_%S_.h5")    
    model_path = os.path.join(model_dir, fileName)
    print(f"your model will be saved at the following location\n{model_path}")
    return model_path

In [94]:
model.save(saveModel_path())

your model will be saved at the following location
SAVED_MODELS\Model_2023_02_17_13_19_45_.h5


In [62]:
def build_model(hp):
    model = Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(
        optimizer=optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_squared_error')
        
    return model

In [66]:
tuner=RandomSearch(build_model,objective='val_loss',max_trials=5,directory='project',project_name='price-pred')

In [67]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 11s]
val_loss: 3759572992.0

Best val_loss So Far: 3759572992.0
Total elapsed time: 00h 00m 46s
INFO:tensorflow:Oracle triggered exit


In [68]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 15,
 'units_0': 96,
 'units_1': 160,
 'learning_rate': 0.01,
 'units_2': 512,
 'units_3': 96,
 'units_4': 256,
 'units_5': 352,
 'units_6': 96,
 'units_7': 512,
 'units_8': 192,
 'units_9': 384,
 'units_10': 384,
 'units_11': 352,
 'units_12': 288,
 'units_13': 352,
 'units_14': 64,
 'units_15': 448,
 'units_16': 256,
 'units_17': 480,
 'units_18': 288,
 'units_19': 192}

In [69]:
mymodel=tuner.get_best_models(num_models=1)[0]

In [70]:
y_pred=mymodel.predict(X_test)

7/7 [==============================] - 1s 5ms/step


In [71]:
y_pred[0:6]

array([[ 12879.607],
       [ 11681.838],
       [ 10090.207],
       [100590.55 ],
       [ 13459.764],
       [ 23925.676]], dtype=float32)

In [72]:
y_test[0:6]

998     56990
304     63350
271     51018
625    159990
951     68999
353     21833
Name: price, dtype: int64